In [ ]:
!pip -q install -U pandas==2.2.2 pyarrow==21.0.0
!pip -q install -U transformers accelerate sentence-transformers scikit-learn datasets

import pandas, pyarrow, transformers, datasets, sklearn, sentence_transformers, accelerate
print("pandas:", pandas.__version__)
print("pyarrow:", pyarrow.__version__)
print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("scikit-learn:", sklearn.__version__)
print("sentence-transformers:", sentence_transformers.__version__)
print("accelerate:", accelerate.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 21.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 20.6 MB/s eta 0:00:00
pandas: 2.2.2
pyarrow: 21.0.0
transformers: 4.56.2
datasets: 4.1.1
scikit-l

In [ ]:
from google.colab import drive
USE_DRIVE = False   # change to False if you don’t want Drive
drive.mount('/content/drive')

if USE_DRIVE:
    SAVE_DIR = "/content/drive/MyDrive/Dataset/colab_runs"
else:
    SAVE_DIR = "/content/baitbuster_runs"

import os
os.makedirs(SAVE_DIR, exist_ok=True)
print("Saving to:", SAVE_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saving to: /content/baitbuster_runs


In [ ]:
import pandas as pd, os

CSV_PATH     = "/content/drive/MyDrive/Dataset/BaitBuster-Bangla_253070_18c_HL10k_AIL.csv"
PARQUET_PATH = "/content/drive/MyDrive/Dataset/BaitBuster-Bangla_253070_18c_HL10k_AIL.parquet"
XLSX_PATH    = "/content/drive/MyDrive/Dataset/BaitBuster-Bangla_253070_18c_HL10k_AIL.xlsx"

if os.path.exists(CSV_PATH):
    df_all = pd.read_csv(CSV_PATH)
elif os.path.exists(PARQUET_PATH):
    df_all = pd.read_parquet(PARQUET_PATH)
elif os.path.exists(XLSX_PATH):
    df_all = pd.read_excel(XLSX_PATH)
else:
    raise FileNotFoundError("Upload BaitBuster dataset and update path!")

print(df_all.columns)
print("Total rows:", len(df_all))

Index(['channel_id', 'channel_name', 'channel_url', 'video_id', 'publishedAt',
       'title', 'title_debiased', 'description', 'description_debiased', 'url',
       'viewCount', 'commentCount', 'likeCount', 'dislikeCount', 'thumbnail',
       'auto_labeled', 'human_labeled', 'ai_labeled'],
      dtype='object')
Total rows: 253070


In [ ]:
import numpy as np

lab = df_all["human_labeled"].astype(str).str.strip().str.lower()

# Normalize variants
lab = lab.replace({
    "not_clickbait": "not clickbait",
    "non-clickbait": "not clickbait",
    "non_clickbait": "not clickbait",
    "notclickbait":  "not clickbait",
    "click bait":    "clickbait",
})

map_dict = {"clickbait": 1, "not clickbait": 0}
mask = lab.isin(map_dict.keys())

df = df_all.loc[mask, ["title_debiased", "human_labeled"]].copy()
df["label"] = lab.loc[mask].map(map_dict).astype(int)
df = df[df["title_debiased"].astype(str).str.strip().ne("")].copy()

print("Human-labeled subset:", len(df))
print("Label distribution:", np.bincount(df["label"]))
df.head()

Human-labeled subset: 10000
Label distribution: [4356 5644]


,title_debiased,human_labeled,label
0,এইমাত্র! মসজিদে নামাজরত অবস্থায় তিন বৃদ্ধকে পি...,Clickbait,1
1,"১০ বছরের সন্তান ফেলে আ,লীগ নেতার সাথে পালিয়ে গ...",Clickbait,1
2,এই মাত্র পাওয়া খবর! ৫ বছরের জেল হচ্ছে পরীমনির!...,Clickbait,1
3,ছি ছি! ভাগিনার সাথে পরকীয়ার সময় হাতেনাতে ধরা খ...,Clickbait,1
4,হায়রে পরীমনি! কারাগারে গিয়েও ভালো হলোনা! কারাগ...,Clickbait,1


In [ ]:
from sklearn.model_selection import train_test_split

X = df["title_debiased"].astype(str).tolist()
y = df["label"].values

X_train, X_tmp, y_train, y_tmp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42
)

print("Train:", np.bincount(y_train))
print("Val  :", np.bincount(y_val))
print("Test :", np.bincount(y_test))

Train: [2614 3386]
Val  : [ 871 1129]
Test : [ 871 1129]


In [ ]:
from datasets import Dataset

def make_hf_ds(texts, labels):
    return Dataset.from_dict({"text": texts, "label": labels})

train_ds = make_hf_ds(X_train, y_train)
val_ds   = make_hf_ds(X_val,   y_val)
test_ds  = make_hf_ds(X_test,  y_test)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score, cohen_kappa_score, confusion_matrix, classification_report
import numpy as np, pandas as pd
import torch

def compute_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "kappa": cohen_kappa_score(y_true, y_pred),
    }

def run_model(model_name: str, run_tag: str, max_length: int = 96, epochs: int = 4, lr: float = 2e-5):
    tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    def tok_fn(ex):
        return tok(ex["text"], truncation=True, padding="max_length", max_length=max_length)
    train_t = train_ds.map(tok_fn, batched=True)
    val_t   = val_ds.map(tok_fn, batched=True)
    test_t  = test_ds.map(tok_fn, batched=True)

    cols = ["input_ids","attention_mask","label"]
    if "token_type_ids" in train_t.features:
        cols.append("token_type_ids")
    for d in (train_t, val_t, test_t):
        d.set_format(type="torch", columns=cols)

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Handle old/new transformers versions
    try:
        args = TrainingArguments(
            output_dir=f"{SAVE_DIR}/{run_tag}",
            learning_rate=lr,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=64,
            num_train_epochs=epochs,
            weight_decay=0.01,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1_macro",
            greater_is_better=True,
            seed=42,
            fp16=torch.cuda.is_available(),
            report_to=[]
        )
    except TypeError:
        args = TrainingArguments(
            output_dir=f"{SAVE_DIR}/{run_tag}",
            learning_rate=lr,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=64,
            num_train_epochs=epochs,
            weight_decay=0.01,
            eval_strategy="epoch",   # older param name
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1_macro",
            greater_is_better=True,
            seed=42,
            fp16=torch.cuda.is_available(),
            report_to=[]
        )

    def hf_metrics(eval_pred):
        logits, labels = eval_pred
        preds = logits.argmax(axis=1)
        return compute_metrics(labels, preds)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_t,
        eval_dataset=val_t,
        compute_metrics=hf_metrics,
    )

    trainer.train()

    # Test evaluation
    raw = trainer.predict(test_t)
    preds = raw.predictions.argmax(axis=1)
    y_true = raw.label_ids
    metrics = compute_metrics(y_true, preds)

    print(f"\n=== {run_tag} RESULTS ===")
    print(metrics)
    print("\nConfusion Matrix:\n", confusion_matrix(y_true, preds))
    print("\nClassification Report:\n", classification_report(y_true, preds, digits=4))

    # Save results
    pd.DataFrame([metrics]).to_csv(f"{SAVE_DIR}/metrics_{run_tag}.csv", index=False)
    return metrics

In [ ]:
models = {
    "MiniLM-L12-v2": "sentence-transformers/all-MiniLM-L12-v2",
    "mpnet-base-v2": "sentence-transformers/all-mpnet-base-v2",
    "xlm-r-v1":      "sentence-transformers/paraphrase-xlm-r-multilingual-v1",
}

results = {}
for tag, name in models.items():
    print("\n" + "="*40)
    print("Running:", tag)
    print("="*40)
    results[tag] = run_model(name, tag, max_length=96, epochs=4, lr=2e-5)

import json
print("\nFinal results:")
print(json.dumps(results, indent=2))


Running: MiniLM-L12-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-xlm-r-multilingual-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,Kappa
1,No log,0.058679,0.981500,0.981174,0.981500,0.962349
2,No log,0.051826,0.985000,0.984722,0.985000,0.969444
3,0.076400,0.057817,0.987500,0.987263,0.987500,0.974527
4,0.076400,0.052311,0.987000,0.986759,0.987000,0.973518



=== xlm-r-v1 RESULTS ===
{'accuracy': 0.991, 'f1_macro': 0.9908354411818615, 'f1_micro': 0.991, 'kappa': 0.9816713489581689}

Confusion Matrix:
 [[ 857   14]
 [   4 1125]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9954    0.9839    0.9896       871
           1     0.9877    0.9965    0.9921      1129

    accuracy                         0.9910      2000
   macro avg     0.9915    0.9902    0.9908      2000
weighted avg     0.9910    0.9910    0.9910      2000


Final results:
{
  "MiniLM-L12-v2": {
    "accuracy": 0.977,
    "f1_macro": 0.9765984052321897,
    "f1_micro": 0.977,
    "kappa": 0.9531970009452171
  },
  "mpnet-base-v2": {
    "accuracy": 0.978,
    "f1_macro": 0.9776158658742683,
    "f1_micro": 0.978,
    "kappa": 0.955231913947599
  },
  "xlm-r-v1": {
    "accuracy": 0.991,
    "f1_macro": 0.9908354411818615,
    "f1_micro": 0.991,
    "kappa": 0.9816713489581689
  }
}


In [ ]:
# Add Bangla-specific BERTs and run with same settings
bangla_models = {
    "banglabert-csebuet": "csebuetnlp/banglabert",
    # Uncomment the next line if you want to compare a second BanglaBERT:
    # "banglabert-sagor":  "sagorsarker/bangla-bert-base",
}

bangla_results = {}
for tag, name in bangla_models.items():
    print("\n" + "="*40)
    print("Running:", tag)
    print("="*40)
    bangla_results[tag] = run_model(name, tag, max_length=96, epochs=4, lr=2e-5)

# Merge and print all results together (paper models + BanglaBERT)
try:
    # 'results' was created in Cell 8
    results.update(bangla_results)
    final_results = results
except NameError:
    final_results = bangla_results

import json
print("\nAll results (incl. BanglaBERT):")
print(json.dumps(final_results, indent=2))



Running: banglabert-csebuet


tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,Kappa
1,No log,0.047780,0.987500,0.987287,0.987500,0.974574
2,No log,0.056680,0.985500,0.985216,0.985500,0.970436
3,0.079700,0.053377,0.987000,0.986783,0.987000,0.973567
4,0.079700,0.049250,0.988500,0.988298,0.988500,0.976595


model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]


=== banglabert-csebuet RESULTS ===
{'accuracy': 0.99, 'f1_macro': 0.9898280948021565, 'f1_micro': 0.99, 'kappa': 0.9796562102978229}

Confusion Matrix:
 [[ 860   11]
 [   9 1120]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9896    0.9874    0.9885       871
           1     0.9903    0.9920    0.9912      1129

    accuracy                         0.9900      2000
   macro avg     0.9900    0.9897    0.9898      2000
weighted avg     0.9900    0.9900    0.9900      2000


All results (incl. BanglaBERT):
{
  "MiniLM-L12-v2": {
    "accuracy": 0.977,
    "f1_macro": 0.9765984052321897,
    "f1_micro": 0.977,
    "kappa": 0.9531970009452171
  },
  "mpnet-base-v2": {
    "accuracy": 0.978,
    "f1_macro": 0.9776158658742683,
    "f1_micro": 0.978,
    "kappa": 0.955231913947599
  },
  "xlm-r-v1": {
    "accuracy": 0.991,
    "f1_macro": 0.9908354411818615,
    "f1_micro": 0.991,
    "kappa": 0.9816713489581689
  },
  "banglabert-cseb